In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.display import display
import sys
import os
import logging

sys.path.insert(0,'../src')
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 100

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from io_utils import save_data_to_file, load_data_from_file
from general_utils import evaluate_rejections
from scGCO import *

def read_trial_data(sim_dir, entry):
    lfn = os.path.join(sim_dir, entry['locs_fn'])
    locs = load_data_from_file(lfn, 'csv')
    dfn = os.path.join(sim_dir, entry['expr_fn'])
    data = load_data_from_file(dfn, 'csv')
    return locs, data
    
# sim_dir = '/scratch/PI/sabatti/space_comp_sim/20191030'
sim_dir = '/share/PI/sabatti/feat_viz/space_comp_sim/20191104'
sim_fn = os.path.join(sim_dir, 'sim_setup.csv')
sim_df = load_data_from_file(sim_fn, 'csv')
display(sim_df.head())

alpha = 0.05
meth_dir = 'result_scgco'
cache=True

meth_dir = os.path.join(sim_dir, meth_dir)
os.makedirs(meth_dir, exist_ok=True)

eval_fn = 'eval_scgco.csv'
eval_fn = os.path.join(sim_dir, eval_fn)
eval_df = pd.DataFrame()

for i, entry in sim_df.iterrows():
    locs, data = read_trial_data(sim_dir, entry)
    assert locs.shape[0] == data.shape[0], 'Mismatch samples'
    data_norm = data.copy()
    fn = os.path.join(meth_dir, 'result_{}.csv'.format(i))
    print(fn)
    if cache and os.path.exists(fn):
        result = load_data_from_file(fn, 'csv')
        rej_idx = result.loc[result['fdr'] < alpha]
    else:
        sf = estimate_smooth_factor(locs.values, 
                                    data_norm, 
                                    start_sf = 20, fdr_cutoff=0.01,iterations=2)
        print('Smooth factor: {}'.format(sf))
        result_df=identify_spatial_genes(locs.values, data_norm, smooth_factor=sf)
        result = result_df[['exp_p_value', 'exp_fdr', 'exp_diff', 'fdr', 'smooth_factor']]
        # save result to file
        save_data_to_file(result, fn, 'csv')        
    # evaluate for tracking
    rej_idx = np.array(result.loc[result['fdr'] < 0.05].index).astype(int)
    nn_idx = np.arange(entry['n_per_reg'] * entry['n_regs'])
    evals = evaluate_rejections(set(rej_idx), set(nn_idx))
    print('{}-{}: Power: {:.4f}, FDP: {:.4f}'.format(entry['temp'], entry['seed'], 
                                              evals['Power'], evals['FDP']))
    eval_df = eval_df.append(pd.Series(evals, name=i))
save_data_to_file(eval_df, eval_fn, 'csv')

,seed,temp,n_per_reg,n_regs,locs_fn,expr_fn
0,0,linear,25,4,locs_2d.csv,sim_data/mtx_linear_0.csv
1,0,diaquad,25,4,locs_2d.csv,sim_data/mtx_diaquad_0.csv
2,0,radial,25,4,locs_2d.csv,sim_data/mtx_radial_0.csv
3,0,cosine,25,4,locs_2d.csv,sim_data/mtx_cosine_0.csv
4,1,linear,25,4,locs_2d.csv,sim_data/mtx_linear_1.csv


/scratch/PI/sabatti/space_comp_sim/20191104/result_scgco/result_0.csv


100%|██████████| 24/24 [00:28<00:00,  1.18s/it]


========iteration1
25 60


100%|██████████| 24/24 [00:28<00:00,  1.19s/it]


========iteration2
30 57
Smooth factor: 30


100%|██████████| 24/24 [00:32<00:00,  1.37s/it]


linear-0: Power: 0.5700, FDP: 0.0000


AssertionError: Mismatch samples

In [6]:
data_norm.shape

(1045, 200)

In [3]:
from io_utils import save_data_to_file, load_data_from_file
from general_utils import norm_mtx, evaluate_rejections

def read_trial_data(sim_dir, entry):
    lfn = os.path.join(sim_dir, entry['locs_fn'])
    locs = load_data_from_file(lfn, 'csv')
    dfn = os.path.join(sim_dir, entry['expr_fn'])
    data = load_data_from_file(dfn, 'csv')
    return locs, data
    
sim_dir = '/scratch/PI/sabatti/space_comp_sim/20191030'
sim_fn = os.path.join(sim_dir, 'sim_setup.csv')
sim_df = load_data_from_file(sim_fn, 'csv')
display(sim_df.head())

meth_dir = 'result_ours'
cache=True

meth_dir = os.path.join(sim_dir, meth_dir)
os.makedirs(meth_dir, exist_ok=True)

eval_fn = 'eval_ours.csv'
eval_fn = os.path.join(sim_dir, eval_fn)
eval_df = pd.DataFrame()


alpha= 0.05
knn = 8
n_perm = 10000

for i, entry in sim_df.iterrows():
    locs, data_norm = read_trial_data(sim_dir, entry)
    data_norm = norm_mtx(data_norm)
    assert locs.shape[0] == data_norm.shape[0], 'Mismatch samples'
    
    fn = os.path.join(meth_dir, 'result_{}.csv'.format(i))
    if cache and os.path.exists(fn):
        result = load_data_from_file(fn, 'csv')
        rej_idx = result.loc[result['reject']].index
    else:
       
        # save result to file
        save_data_to_file(result, fn, 'csv')        
    # evaluate for tracking
    nn_idx = np.arange(entry['n_per_reg'] * entry['n_regs'])
    evals = evaluate_rejections(set(rej_idx), set(nn_idx))
    print('{}-{}: Power: {:.4f}, FDP: {:.4f}'.format(entry['temp'], entry['seed'], 
                                              evals['Power'], evals['FDP']))
    
    eval_df = eval_df.append(pd.Series(evals, name=i))
save_data_to_file(eval_df, eval_fn, 'csv')

,seed,temp,n_per_reg,n_regs,locs_fn,expr_fn
0,0,linear,25,4,locs_2d.csv,sim_data/mtx_linear_0.csv
1,0,diaquad,25,4,locs_2d.csv,sim_data/mtx_diaquad_0.csv
2,0,radial,25,4,locs_2d.csv,sim_data/mtx_radial_0.csv
3,0,cosine,25,4,locs_2d.csv,sim_data/mtx_cosine_0.csv
4,1,linear,25,4,locs_2d.csv,sim_data/mtx_linear_1.csv


linear-0: Power: 1.0000, FDP: 0.0291
diaquad-0: Power: 0.9700, FDP: 0.0300
radial-0: Power: 1.0000, FDP: 0.0196
cosine-0: Power: 0.7600, FDP: 0.0617
linear-1: Power: 1.0000, FDP: 0.0099


AssertionError: Mismatch samples

In [7]:
locs.shape

(1045, 2)